In [1]:
%pylab inline

import os.path as op
import sys
import matplotlib.pyplot as plt
import glob

import io3d
import sed3


sys.path.append(op.expanduser("~/projects/bodynavigation"))

import bodynavigation

sys.path.append(op.expanduser("~/projects/imtools"))
import imtools
reload(imtools)

Populating the interactive namespace from numpy and matplotlib


ImportError: No module named imtools

In [2]:
sliver_reference_dir = op.expanduser("~/data/medical/orig/sliver07/training/")

## Train

Prepare function to calculate feature vector from 3D to [1D x number_of_features]

In [3]:
def localization_fv(data3d, voxelsize_mm):        # scale
        fv = []
        # f0 = scipy.ndimage.filters.gaussian_filter(data3d, sigma=3).reshape(-1, 1)
        #f1 = scipy.ndimage.filters.gaussian_filter(data3dr, sigma=1).reshape(-1, 1) - f0
        #f2 = scipy.ndimage.filters.gaussian_filter(data3dr, sigma=5).reshape(-1, 1) - f0
        #f3 = scipy.ndimage.filters.gaussian_filter(data3dr, sigma=10).reshape(-1, 1) - f0
        #f4 = scipy.ndimage.filters.gaussian_filter(data3dr, sigma=20).reshape(-1, 1) - f0
        # position asdfas
        import bodynavigation as bn
        ss = bn.BodyNavigation(data3d, voxelsize_mm)
        fd1 = ss.dist_to_lungs().reshape(-1, 1)
        fd2 = ss.dist_to_spine().reshape(-1, 1)
        fd3 = ss.dist_sagittal().reshape(-1, 1)
        fd4 = ss.dist_coronal().reshape(-1, 1)
        fd5 = ss.dist_axial().reshape(-1, 1)
        fd6 = ss.dist_to_surface().reshape(-1, 1)
        fd7 = ss.dist_diaphragm().reshape(-1, 1)

        # f6 = scipy.ndimage.filters.gaussian_filter(data3d, sigma=[20, 1, 1]).reshape(-1, 1) - f0
        # f7 = scipy.ndimage.filters.gaussian_filter(data3d, sigma=[1, 20, 1]).reshape(-1, 1) - f0
        # f8 = scipy.ndimage.filters.gaussian_filter(data3d, sigma=[1, 1, 20]).reshape(-1, 1) - f0

        # print "fv shapes ", f0.shape, fd2.shape, fd3.shape
        fv = np.concatenate([
                # f0,
#                 f1, f2, f3, f4,
                fd1, fd2, fd3, fd4, fd5, fd6, fd7,
                #f6, f7, f8
            ], 1)


        return fv

In [4]:
# import lisa.organ_localizator

# ol = lisa.organ_localizator.OrganLocalizator()
import imtools.trainer3d
ol = imtools.trainer3d.Trainer3D()
ol.feature_function = localization_fv

for one in imtools.datasets.sliver_reader("*[0].mhd", read_seg=True):
    numeric_label, vs_mm, oname, orig_data, rname, ref_data = one
    ol.add_train_data(orig_data, ref_data, voxelsize_mm=vs_mm)
    
ol.fit()

/home/mjirik/miniconda/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/mjirik/projects/bodynavigation/bodynavigation/body_navigation.py:398: RuntimeWarning: invalid value encountered in greater
  med = np.median(profile[profile > 0])
/home/mjirik/projects/bodynavigation/bodynavigation/body_navigation.py:399: RuntimeWarning: invalid value encountered in greater
  profile[np.abs(profile - med) > tolerance] = 0
/home/mjirik/miniconda/lib/python2.7/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
/home/mjirik/projects/bodynavigation/bodynavigation/body_navigation.py:506: RuntimeWarning: invalid value encountered in greater
  too_bad_pixels = np.abs(flat_out - flat) > (max_dist_

## Testing

In [5]:
one = list(sliver_reader("*018.mhd", read_seg=True))[0]
numeric_label, vs_mm, oname, orig_data, rname, ref_data = one
fit = ol.predict(orig_data, voxelsize_mm=vs_mm)

ValueError: total size of new array must be unchanged

In [ ]:
plt.figure(figsize=(15,10))
sed3.show_slices(orig_data, fit, slice_step=20, axis=1, flipV=True)

### Evaluation

In [ ]:
import lisa.volumetry_evaluation
lisa.volumetry_evaluation.compare_volumes_sliver(ref_data, fit, vs_mm)